<a href="https://colab.research.google.com/github/dhphu19/weatherforecat/blob/main/Copy_of_Weather_forecast_TCN_MISO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!pip install keras-tcn
import wget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tqdm.notebook import tqdm
from tensorflow import keras
from tcn import TCN,  tcn_full_summary
from sklearn import preprocessing
import math
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
file_path = '/content/gdrive/MyDrive/weather_data/WeatherData_Aug_2020.csv'
data=pd.read_csv(file_path, delimiter=',',skipinitialspace=True)
data.head(24)
#chuan hoa
data2 = data[['Temperature','Wind Speed','Relative Humidity','Visibility','Precipitation']] #'Conditions'
dieuChinh = preprocessing.MinMaxScaler(feature_range= (0,1))
tenCot = ['temperature', 'windSpeed', 'humidity', 'visibility', 'precipitation']
data3 = dieuChinh.fit_transform(data2)
Max = dieuChinh.data_max_
Min = dieuChinh.data_min_
print(Max)
print(Min)
# data2.to_csv("data3.csv")
np.savetxt("data3.csv", data3, delimiter = ",")
data4 = pd.read_csv('/content/data3.csv', names=tenCot)
data4

In [ ]:
# data training
temperature = np.array(data4['temperature'])
windSpeed = np.array(data4['windSpeed'])
humidity = np.array(data4['humidity'])
visibility = np.array(data4['visibility'])
precipitation = np.array(data4['precipitation'])

dataInput = np.array([temperature, humidity, windSpeed, visibility, precipitation]) # gop 5 ma tran lam mot
dataInput = np.transpose(dataInput) # chuyen vi ma tran
num_periods = 24
f_horizon = 1
x_data = dataInput[:(len(dataInput)-(num_periods*2))] 
x_batches = x_data.reshape(-1, num_periods, 5)

y_data = temperature[1:(len(temperature)-(num_periods*2))+f_horizon] #########
y_batches = y_data.reshape(-1, num_periods, 1)
print(x_data.shape)
print(x_batches.shape)
print(y_data.shape)
print(y_batches.shape)

In [ ]:
# data test
def test_data(series_1, series_2, forecast, num):
    testX = series_1[-(num + forecast):][:num].reshape(-1, num_periods, 5)
    testY = series_2[-(num):].reshape(-1, num_periods, 1)
    return testX, testY
X_test, Y_test = test_data(dataInput, temperature, f_horizon, num_periods*2)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
i = Input(batch_shape=(None, num_periods, 5))
m = TCN(return_sequences=True)(i)
m = Dense(1, activation='linear')(m)
model = Model(inputs=[i], outputs=[m])
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mae')

In [ ]:
#print('Train...')
#hist = model.fit(x_batches, y_batches, epochs=1000)
history = model.fit(x_batches, y_batches, epochs=1000, validation_data=(X_test, Y_test))
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
predict = model.predict(X_test)
print(predict.shape)

In [ ]:
# Inv
# 'temperature': 0, 'windSpeed': 1, 'humidity': 2, 'visibility': 3, 'precipitation': 4
Y_test = Y_test*(Max[0]-Min[0])+Min[0]
Y_test = np.ravel(Y_test)
predict = predict*(Max[0]-Min[0])+Min[0]
predict = np.ravel(predict)
np.savetxt("test.txt", predict, delimiter = ",")

In [ ]:
plt.title("forecast vs actual", fontsize=14)
plt.plot(pd.Series(Y_test), "b-", markersize=10, label="Actual")
plt.plot(pd.Series(predict), "r-", markersize=10, label="Forecast")
plt.legend(loc="upper left")
plt.xlabel("Time Periods")
plt.show()

In [ ]:
#in sai số
Y_test = Y_test.reshape(1, -1)
y_pred = predict.reshape(1, -1)
error = Y_test - y_pred
print(max(max(abs(error))))
print(min(max(abs(error))))
print(abs(error))

In [ ]:
# in rmse
rmse = math.sqrt(mean_squared_error(Y_test, y_pred))
print(rmse)